## Parse and extract text from `warc` file
1. With `fastwarc` and `resiliparse`
    1. https://resiliparse.chatnoir.eu/en/stable/index.html

In [ ]:
from fastwarc import ArchiveIterator
from resiliparse.extract.html2text import extract_plain_text
from resiliparse.parse.encoding import detect_encoding

def extract_text(record):
    byte_string = record.reader.read()
    encoding = detect_encoding(byte_string)
    html_content = byte_string.decode(encoding=encoding)
    extracted_text = extract_plain_text(html_content)
    return extracted_text

warc_file = "/home/azureuser/localfiles/cs336-assignment4-data-mine/cs336_data/CC-MAIN-20250417135010-20250417165010-00065.warc.gz"
iterator = ArchiveIterator(open(warc_file, "rb"), func_filter=lambda r: r.headers.get('WARC-Identified-Payload-Type') == 'text/html')

# record = next(iterator)
# print(extract_text(record))

In [ ]:
# To get random records for ensuing tests
# Method: Skip randomly through iterator
import random

def get_random_records(iterator, N=20, skip_prob=0.9):
    """Skip records randomly and return extracted text immediately"""
    random_data = []
    for i, record in enumerate(iterator):
        if len(random_data) >= N:
            break
        if random.random() > skip_prob:
            # Read and process immediately while record is still fresh
            extracted_text = extract_text(record)
            random_data.append((i, extracted_text))
    return random_data

## Language identification

In [ ]:
import fasttext
model = fasttext.load_model("lid.176.bin")

# sanity check
model.predict("Hello world.")

In [ ]:
def language_generator():
    for i, text in random_data:
        print(f"=== Record {i} ===")
        print(text[:200] + "..." if len(text) > 200 else text)
        lang, score = model.predict(text.replace("\n", " "))
        print(f"Language: {lang[0]}, Score: {score[0]:.4f}")
        print("-" * 50)
        yield 

# Get random records with extracted text
random_data = get_random_records(iterator)

# Create the generator
lang_gen = language_generator()


In [ ]:
# Run this cell multiple times to get one result at a time
next(lang_gen)

## PII
### email address
- Length: The total length of an email address is capped at 320 characters, with 64 for the username and 255 for the domain.
- Spaces: Spaces are not allowed.
- Case sensitivity: Email addresses are generally not case-sensitive, meaning User@Example.com is the same as user@example.com.

- Special characters:
    - Periods (.), hyphens (-), and underscores (_) are often allowed in the local part.
    - They cannot be the first or last character of the local part and cannot appear consecutively (e.g., john..doe@example.com is invalid).
    - In the domain, hyphens are allowed but not at the beginning or end of a label (a part between periods). 

### US phone number
1. use pattern
    - (\+1\s*)? - optional `+1` followed by optional spaces
    - \(? - optional opening parenthesis
    - \d{3} - 3 digits
    - \)? - optional closing parenthesis
    - [\s-]? - at most one space or hyphen (the ? means zero or one)
    - \s* - zero or more additional spaces
    - \d{3} - 3 digits
    - [\s-]? - at most one space or hyphen
    - \s* - zero or more additional spaces
    - \d{4} - 4 digits

### IP address
- Use `\b` to avoid matching things like:
    - 1.2.3.4.5 (too many octets)
    - version1.2.3.4 (prefix attached)
    - 1.2.3.4th (suffix attached)

In [ ]:
from mask_pii import mask_email, mask_phone, mask_ip

record = next(iterator)
text = extract_text(record)
print(text)

In [ ]:
text, mask_email_counts = mask_email(text)
text, mask_phone_counts = mask_phone(text)
text, mask_ip_counts = mask_ip(text)

if any([counts > 0 for counts in [mask_email_counts, mask_phone_counts, mask_ip_counts]]):
    print(text)

## Harmful content

In [ ]:
import fasttext

model = fasttext.load_model("/home/azureuser/localfiles/cs336-assignment4-data-mine/cs336_data/jigsaw_fasttext_bigrams_nsfw_final.bin")


In [ ]:
text = (
    "SUCK MY C*CK WIKIPEDIA EDITORS...F*CKING *SSH*LE DORKS. "
    "JUST TRYING TO MAKE THE SITE BETTER YOU UPTIGHT C*NTS"
)
# text = "Umm, theres no actual article for prostitution ring.  - Crunch Captain."
text = "操你妈 你个傻逼 干死你"

model.predict(text)

In [ ]:
print(text)

In [ ]:
from harmful_content import classify_nsfw, classify_toxic_speech

found = False
while found == False:
    record = next(iterator)
    try:
        text = extract_text(record)
    except UnicodeDecodeError:
        continue
    # print(text)
    is_nsfw, score_nsfw = classify_nsfw(text)
    is_toxic, score_toxic = classify_toxic_speech(text)

    if is_nsfw == "nsfw" or is_toxic == "toxic":
        print(text)
        print("="*80)
        print("Harmful content detected!")
        print(is_nsfw, is_toxic)
        print(score_nsfw, score_toxic)

        found = True


## Quality Classifier

- To get `warc` file
    - `wget --tries=2 --timeout=5 -i enwiki-20240420-extracted_urls_subset.txt --warc-file=subsampled_positive_urls -O /dev/null`

In [ ]:
from fastwarc import ArchiveIterator
from fastwarc.warc import WarcRecord
from resiliparse.extract.html2text import extract_plain_text
from resiliparse.parse.encoding import detect_encoding

def extract_text(record):
    byte_string = record.reader.read()
    encoding = detect_encoding(byte_string)
    try:
        html_content = byte_string.decode(encoding=encoding)
    except UnicodeDecodeError:
        return ""
    extracted_text = extract_plain_text(html_content)
    return extracted_text


In [ ]:
def is_html_record(record: WarcRecord) -> bool:
    try:
        return record.http_headers.get('Content-Type', '').startswith('text/html')
    except:
        return False

warc_file = "/home/azureuser/localfiles/cs336-assignment4-data-mine/cs336_data/subsampled_positive_urls.warc.gz"
iterator = ArchiveIterator(open(warc_file, "rb"))

from language_identification import identify_language
positive_texts = []
for record in iterator:
    if is_html_record(record):
        text = extract_text(record)
        if len(text) > 512:
            lang, score = identify_language(text)
            if lang == "en" and score > 0.8:
                positive_texts.append(text)

In [ ]:
import random
from language_identification import identify_language

def get_random_english_records(iterator, N=600, skip_prob=0.8):
    """Skip records randomly and return extracted text immediately"""
    random_data = []
    for record in iterator:
        if len(random_data) >= N:
            break
        if random.random() > skip_prob:
            # Read and process immediately while record is still fresh
            extracted_text = extract_text(record)
            lang, score = identify_language(extracted_text)
            if lang == "en" and score > 0.8:
                random_data.append(extracted_text)
    return random_data

In [ ]:
warc_file = "/home/azureuser/localfiles/cs336-assignment4-data-mine/cs336_data/CC-MAIN-20250417135010-20250417165010-00065.warc.gz"
iterator = ArchiveIterator(open(warc_file, "rb"), func_filter=lambda r: r.headers.get('WARC-Identified-Payload-Type') == 'text/html')

In [ ]:
negative_texts = get_random_english_records(iterator)

In [ ]:
import pickle

# Save to pickle file
with open('positive_texts.pkl', 'wb') as f:
    pickle.dump(positive_texts, f)
with open('negative_texts.pkl', 'wb') as f:
    pickle.dump(negative_texts, f)


In [ ]:
from datasets import Dataset

labels = [0] * 400 + [1] * 400
texts = negative_texts[:400] + positive_texts[:400]

ds = Dataset.from_dict({"text": texts, "label": labels})

In [ ]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)